In [1]:
import csv
import pandas as pd

In [2]:
filenames = ['colors.csv', 'inventories.csv', 'inventory_parts.csv', 'inventory_sets.csv', 'part_categories.csv', 'parts.csv', 'sets.csv', 'themes.csv']

with open(filenames[0]) as datafile:
    colors = pd.read_csv(datafile)
with open(filenames[1]) as datafile:
    inventories = pd.read_csv(datafile)
with open(filenames[2]) as datafile:
    inventory_parts = pd.read_csv(datafile)
with open(filenames[3]) as datafile:
    inventory_sets = pd.read_csv(datafile)
with open(filenames[4]) as datafile:
    part_categories = pd.read_csv(datafile)
with open(filenames[5]) as datafile:
    parts = pd.read_csv(datafile)
with open(filenames[6]) as datafile:
    sets = pd.read_csv(datafile)
with open(filenames[7]) as datafile:
    themes = pd.read_csv(datafile)

In [3]:
sets = pd.merge(sets, themes, left_on = 'theme_id', right_on = 'id')

In [8]:
sets

,setname,year,theme_id,num_parts,theme_name,parent_id
set_num,,,,,,
00-1,Weetabix Castle,1970,414,471,Castle,411.0
0011-2,Town Mini-Figures,1978,84,12,Supplemental,67.0
2181-1,Infomaniac,1997,84,4,Supplemental,67.0
300-1,T-Junction Road Plates,1978,84,2,Supplemental,67.0
301-1,Curved Road Plates,1978,84,2,Supplemental,67.0
302-1,Straight Road Plates (without crosswalk),1983,84,2,Supplemental,67.0
302-2,Straight Road Plates (with crosswalk),1978,84,2,Supplemental,67.0
551-1,T Road Plates,1979,84,2,Supplemental,67.0
552-1,Curved Road Plates,1979,84,2,Supplemental,67.0


In [4]:
sets = sets.drop('id', axis = 1)

In [5]:
sets = sets.set_index(['set_num'])

In [6]:
sets.columns = ['setname', 'year', 'theme_id', 'num_parts', 'theme_name', 'parent_id']

In [7]:
sets[sets['year']==1990]

,setname,year,theme_id,num_parts,theme_name,parent_id
set_num,,,,,,
1520-2,Town 2 for 1 Bonus Offer,1990,67,2,Classic Town,50.0
1581-2,Delivery Truck,1990,75,122,Food & Drink,67.0
6376-1,Breezeway Cafe (Breezeway Café),1990,75,194,Food & Drink,67.0
1876-1,Soil Scooper,1990,85,73,Traffic,67.0
6530-1,Sport Coupe,1990,85,43,Traffic,67.0
6644-1,Road Rebel,1990,85,72,Traffic,67.0
1677-1,Rabbit,1990,229,25,Easter,206.0
5590-1,Whirl and Wheel Super Truck,1990,278,1063,Model Team,NaN
1560-1,Glory Glider,1990,68,26,Airport,67.0


In [ ]:
inventory_parts